In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
import plotly.express as px
import calendar
def my_formatter(x):
    if x < 0:
        return '{:,.0f}'.format(-x)
    return '{:,.0f}'.format(x)
pd.options.display.float_format = my_formatter

In [ ]:
import pandas_profiling

In [ ]:
report = pandas_profiling.ProfileReport(df)
report.to_file(r"C:\Users\kaihsu\Desktop\axmr_report.html")

In [ ]:
df["預交年份"] = df["預交年份"].astype("str")

In [ ]:
chart = df.groupby(["客戶集團", "預交年份"])[["金額*集團匯率(NTD)"]].sum().reset_index()

In [ ]:
px.bar(chart, x="客戶集團", y="金額*集團匯率(NTD)", color="預交年份", barmode="group")

In [ ]:
month_name = [calendar.month_name[i] for i in range(1, 13)]

In [ ]:
month_number = [str(i) for i in range(1, 13)]

In [ ]:
month_dict = dict(zip(month_number, month_name))

In [ ]:
df["預交月份"] = df["預交日期"].str.split("-", expand=True)[1].str.replace("0", "").map(month_dict)

In [ ]:
df.BU.unique()

In [ ]:
df["分類"] = df.apply(lambda x: "元件" if x["BU"] not in ["RF1", "RF2"] else "天線" if x["BU"] in ["RF1", "RF2"] else "未分類", axis=1)

In [ ]:
df.columns

## 整理2022年預算為csv

In [ ]:
budget_2022 = pd.read_excel(r"C:\Users\kaihsu\OneDrive\INPAQ\預算\2022 Forecast\正式預算\20211129_第四版預算\2022年最終版預算.xlsx", sheet_name="Sheet1")

In [ ]:
budget_2022["數量"] = budget_2022["數量"].map(lambda x: 0 if x == " -  " else x).astype("int")

In [ ]:
budget_2022 = budget_2022.astype({"年份": "str", "數量": "int", "金額": "int"})

In [ ]:
budget_2022["Group"] = budget_2022["Group"].str.upper()

In [ ]:
budget_2022.to_csv(r"C:\Users\kaihsu\PycharmProjects\inpaq_streamlit_project\data\budget_2022.csv", index=False)

In [ ]:
budget_2022.columns.tolist()

## 預算和實際一起合併

In [ ]:
revenue = pd.read_csv(r"C:\Users\kaihsu\PycharmProjects\inpaq_streamlit_project\data\axmr_transform_file\converted_axmr430.csv")

In [ ]:
def combine_numbers(revenue, budget):
    columns = ["負責業務", "BG", "Group", "年份", "月份", "數量", "金額", "品名", "類型", "金額類型"]
    trans_rev = revenue.groupby(["訂單業務", "BU", "客戶集團", "預交年份", "預交月份", "品名", "分類"])[["數量", "金額*集團匯率(NTD)"]].sum().reset_index()
    trans_rev["金額類型"] = "實績"
    trans_rev = trans_rev.reindex(columns=["訂單業務", "BU", "客戶集團", "預交年份", "預交月份", "數量", "金額*集團匯率(NTD)", "品名", "分類", "金額類型"],)
    trans_rev.columns = columns
    result = pd.concat([trans_rev, budget], axis=0)
    return result

## 圖表設計

## 顯示每月預算和實績

In [ ]:
pd.pivot_table(chart, index=["負責業務", "類型", "年份", "金額類型"],  columns="月份", values="金額")

## 使用Plotly作圖

In [ ]:
chart = combine_numbers(revenue, budget_2022)

In [ ]:
month_name = [calendar.month_name[i] for i in range(1, 13)]

In [ ]:
chart_filtered = chart.groupby(["年份", "負責業務", "月份", "金額類型"])[["金額"]].sum().reset_index()

In [ ]:
px.area(
chart_filtered.query("負責業務 == '許凱智' & 年份 == '2022'"), x="月份", y="金額", color="金額類型", category_orders={"月份": month_name}
)